
# Fuzzy C-means Clustering

## This program performs Fuzzy C-means clustering on the Flu dataset.


### Mount Google Drive if working on google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Change directory to a path where data is located with respect to the code

In [ ]:
%cd /content/drive/My\ Drive/clustering

/content/drive/My Drive/Courses/Machine Learning


### Install packages

In [ ]:
!pip3 install numpy
!pip3 install pandas
!pip3 install scikit-learn
!pip3 install seaborn
!pip3 install matplotlib
!pip3 install scipy
!pip3 install statistics

  Created wheel for statistics: filename=statistics-1.0.3.5-cp36-none-any.whl size=7454 sha256=900cfb51339441dd56efc957086f3117aa16518fbc5c4b094c567a23d51a3008
  Stored in directory: /root/.cache/pip/wheels/75/55/90/73aa7662bfb4565b567618547a275f01372a678ca92ecd64f3
Successfully built statistics


## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt
from pprint import pprint
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math
from collections import defaultdict
from scipy.spatial import distance
import statistics

%matplotlib inline

## Load dataset

In [ ]:
df = pd.read_csv('Assignment-I/Assignment1_data.csv')   # change to the path where the data is stored

In [ ]:
df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()
df.head()

,Student,Vaccin,HndWshQual,HndWshFreq,SociDist,NoFaceContact,RespEttiqu,PersnDist,HandSanit,Risk,Complications,Barriers,Inefficacy,KnowlTrans,KnowlMgmt,Sick,Flu,Female
0,1,3,4,4,2,1,5,1,1,-0.770,-1.453,0.000,0.929,-0.554,0.000,0.0,0.0,1.0
1,2,2,4,4,5,2,5,4,4,-0.345,0.000,-0.489,0.149,-0.554,1.482,1.0,0.0,0.0
2,3,3,2,2,2,3,2,2,1,-0.406,-0.575,-0.234,0.693,-0.182,-1.482,0.0,0.0,0.0
4,5,2,5,3,3,2,5,5,3,0.000,-0.770,0.097,0.546,0.554,0.684,1.0,0.0,0.0
5,6,2,5,5,3,3,2,3,1,0.169,-0.169,-0.726,0.370,0.951,0.684,2.0,1.0,0.0


### Fuzzy C-means clustering implementation block. 

In [ ]:
# Function to extract features along with the data points
# the function accepts list of features and returns data with only these features
def _extractFeatures(features):
  return df[features].dropna()

# Method to compute dunn index when passed records (with their cluster label) and centroids (with their cluster labels)
def _computeDunnIndex(records, centroids):
  # Compute inter cluster distances----i.e., distances between the centroids
  list_centroids = list(centroids.values())
  smallest_inter_cluster_dist = 1000000  # Assuming smallest inter-cluster distance can't be over 100000
  for idx, current_cent in enumerate(list_centroids):
    for next_cent in list_centroids[idx+1:]:
      dist = distance.euclidean(current_cent, next_cent)
      if dist < smallest_inter_cluster_dist:
        smallest_inter_cluster_dist = dist

  # Computer intra-cluster distances
  # Iterate through the records
  largest_intra_cluster_dist = 0.0
  for cluster_label, points in records.items():
    for point in points:
      dist = distance.euclidean(point,centroids[cluster_label])
      if dist > largest_intra_cluster_dist:
        largest_intra_cluster_dist = dist

  dunn_idx = float(smallest_inter_cluster_dist)/largest_intra_cluster_dist
  return dunn_idx

# Method to compute the sum of squared errors given cluster_dict {cluster-0: (rec-0, w-0), cluster-1: (rec-1, w-1),..., cluster-(k-1): (rec-k-1, w-(k-1))}
# and centroid_dict {cluster-0: centroid-0, cluster-1: centroid-1,..., cluster-k-1: centroid-2}

class FuzzyCMeans:
  def __init__(self, k, p, data, eps):
    self.k = k   # number of clusters
    self.p = p   # fuzzification parameter
    self.data = data  # data to cluster
    self.eps = eps     # threshold to detect if centroids significantly changed position 
  
  # Method to perform Fuzzy C means clustering---accepts number of clusters, p, the data with specific features, and convergence parameter
  def _fuzzyCMeansSSE(self, cluster_dict, centroid_dict):
    sum_over_clusters = 0.0
    for cluster_id, records_w_weights in cluster_dict.items():
      sum_over_records = 0.0
      for rec_weight in records_w_weights:   # rec_weight is a tuple of record-weight
        record = rec_weight[0]
        weight = rec_weight[1]
        centroid = centroid_dict[cluster_id]
        sum_over_records += math.pow(weight, self.p) * distance.euclidean(record, centroid)
      sum_over_clusters += sum_over_records
    
    return sum_over_clusters
  
  def _fuzzyCMeans(self):
    # standardize the data
    data_std = StandardScaler().fit_transform(self.data)
    # randomly initialize w_ij(weight of record i in cluster j) as many as there are number of clusters (i.e., 'k')
    record_dict = defaultdict(list)    # the key is the data point identifier--row index and the value is the list of weights in each cluster (there are 'k' clusters)
    
    for record_idx, record in enumerate(data_std):
      weight_list = np.random.random(self.k)
      weight_list /= weight_list.sum()    # to make sure all weights of a record sum to 1

      # record_dict holds the record associated with the weight_list of the record in a dictionary as a tuple
      record_dict[record_idx] = (record, weight_list)  # all the elements of the list, whose length is equal to the number of clusters, sum upto 1---each index in the list corresponds to cluster label
    
    cluster_dict = defaultdict(list) # key is the cluster label and value is tuples of record-weight
    # assign all points to the respective clusters
    for record_idx, record_w_weights in record_dict.items():
      record = record_w_weights[0]   # the first element of the dict is the record
      
      # iterate through the weight list to grab the weight corresponding to the different clusters
      for cluster_id, weight in enumerate(record_w_weights[1]): 
        # record stays the same in the different clusters---only the weight changes
        cluster_dict[cluster_id].append((record, weight))  # weights across different data points having the same index location in their weight_list are put to the same cluster
    
    minimal_centroid_change = 0  # keeps track of how many centroids havent' significantly changed
    centroid_dict = {}
    # initialize centroids to the first points from cluster_dict
    for idx, records_w_weights in cluster_dict.items():
      centroid_dict[idx] = records_w_weights[0][0] 

    while True:
      # compute centroids---iterate over cluster_dict which has record-weight mappings of all data points assigned to each cluster
      numerator = np.zeros(data_std[0].shape) # the numerator has a form of one of the data points (a record)
      denominator = 0.0
      for cluster_id, records_w_weights in cluster_dict.items():  # weight_list here is the weights of the different data points assigned to a cluster
        for record_weight in records_w_weights:
          record, weight = record_weight[0], record_weight[1]   
          # computing centroids in fuzzy c means is different from k-means
          numerator += record * (math.pow(weight, self.p))
          denominator += math.pow(weight, self.p)

        centroid = numerator / denominator   # computing new centroid for a cluster

        diff = distance.euclidean(centroid_dict[cluster_id], centroid)   # change in newly computed centroid and old centroid
        if diff < self.eps:
          minimal_centroid_change += 1

        if minimal_centroid_change == self.k:   # if minimal_centroid_change counter hits 'k', it means all centroids "stopped" movind and thus all clusters have converged
          return cluster_dict, centroid_dict   #return mapping to the calling program

        centroid_dict[cluster_id] = centroid   # cluster_id to centroid mapping

      # Recompute w_ij for each data point in each cluster
      for cluster_id, records_w_weights in cluster_dict.items():
        centroid = centroid_dict[cluster_id]   # centroid in the current cluster
        
        for record_weight in records_w_weights:
          record = record_weight[0]   # the first item in the tuple is the record

          record_current_centroid_dist = distance.euclidean(record, centroid)   # numerator of the denominator---distance from current centroid(in the current cluster)
          ratio_raised = 0.0  # a value corresponding to a record in a cluster----i.e., with each record, ratio_raised is reset
          # iterate through each cluster and compute distance of the record from the other centroids
          for cluster_id in range(self.k):
            all_centroid = centroid_dict[cluster_id]
            record_all_centroid_dist = distance.euclidean(record, all_centroid)
            ratio = float(record_current_centroid_dist) / record_all_centroid_dist
            
            ratio_raised += math.pow(ratio, 2/(self.p - 1))    # entire denominator in w_ij computation

          weight_updated = 1 / float(ratio_raised)  # This value is then placed along with the record in the relevant dictionaries
          for idx, rec_weight in enumerate(cluster_dict[cluster_id]):
            if (rec_weight[0] == record).all():
              cluster_dict[cluster_id][idx] = (record, weight_updated)

      # Normalizing the weights of the records by first indexing the weights using their record ids
      record_w_weights_list = defaultdict(list)  # key is record identifier and value is list of weights for the record
      for cluster_id, list_records_w_weights in cluster_dict.items():
        for record_idx, record_weight in enumerate(list_records_w_weights):
          record_w_weights_list[record_idx].append(float(record_weight[1]))
          
      # Next normalize these weights that are mapped to their respective records
      # we will then have {rec-1:[normalized_weight-1, normalized_weight-2,...], rec-2:[......]}
      for record_idx, list_weights in record_w_weights_list.items():
        record_w_weights_list[record_idx] = [weight/sum(list_weights) for weight in list_weights]
  
      # update cluster_dict accordingly(i.e., change the weight of each record in each cluster)
      for record_idx, list_weights in record_w_weights_list.items():
        for cluster_id, weight in enumerate(list_weights):
          cluster_dict[cluster_id][record_idx] = (cluster_dict[cluster_id][record_idx][0], weight)

  # Method that returns records and their weights in different clusters when pass the cluster dictionary returned by the 
  # main fuzzy c-means clustering method above
  def _getRecordAndWeights(self, cluster_dict):
    # Extract the records along with their cluster membership and their weights
    record_dict = defaultdict(list)    # this data structure will have the form {cluster_id:[record-1, record-2,..., record-m]}
    for cluster_id, records_w_weights in cluster_dict.items():
      for record_w_weight in records_w_weights:
        record_dict[cluster_id].append(record_w_weight[1])
    return record_dict

### Select features that gave the best combination in K-means clustering when evaluated on Dunn Index

In [ ]:
# Features to extract
features = ['Risk', 'NoFaceContact', 'Sick', 'HandSanit', 'HndWshQual']   # From K-means clustering task

df_cluster = _extractFeatures(features)  # the data to cluster

### Run Fuzzy C-means clustering

In [ ]:
F1 = FuzzyCMeans(2, 2, df_cluster, 0.0005)   # Setting number of clusters to 2 and fuzzification parameter to 2
cluster_dict, centroid_dict = F1._fuzzyCMeans()

### Print the centroids

In [ ]:
cent_0, cent_1 = centroid_dict[0], centroid_dict[1]

print(cent_0)
print(cent_1)

[-0.00942912  0.01140455  0.00255248 -0.0052038   0.00466738]
[ 0.00184234 -0.00346128 -0.00413737  0.00408361  0.00035235]


### Extract the records along with their cluster membership and their weights

In [ ]:
# Extract the records along with their cluster membership and their weights
record_dict = F1._getRecordAndWeights(cluster_dict)    # this data structure will have the form {cluster_id:[record-1, record-2,..., record-m]}

### A sample record along with its weight in each of the clusters computed using the membership function after Fuzzy C-means has run and converged. Look that for a record its weights in different clusters are expected to roughly sum to 1.0

In [ ]:
sse = F1._fuzzyCMeansSSE(cluster_dict, centroid_dict)
print(sse)

372.1778233527125


In [ ]:
record_dict = F1._getRecordAndWeights(cluster_dict)    # {cluster_id:[record-1, record-2,..., record-m]}

### A sample record along with its weight in each of the clusters computed using the membership function after Fuzzy C-means has run and converged. Look that for a record its weights in different clusters are expected to roughly sum to 1.0

In [ ]:
print(record_dict[0][4])
print(record_dict[1][4])

0.5011452594676159
0.498854740532384


In [ ]:
print(_computeDunnIndex(record_dict, centroid_dict))

0.010004843852793125


### Testing Fuzzy C-means clustering with an additional feature (Problem 3c)

In [ ]:
# Select an additional feature from the data except from the 5 features in the previous tasks
df_additional = df.drop(features, axis=1)

In [ ]:
df_additional.head()

,Student,Vaccin,HndWshFreq,SociDist,RespEttiqu,PersnDist,Complications,Barriers,Inefficacy,KnowlTrans,KnowlMgmt,Flu,Female
0,1,3,4,2,5,1,-1.453,0.000,0.929,-0.554,0.000,0.0,1.0
1,2,2,4,5,5,4,0.000,-0.489,0.149,-0.554,1.482,0.0,0.0
2,3,3,2,2,2,2,-0.575,-0.234,0.693,-0.182,-1.482,0.0,0.0
4,5,2,3,3,5,5,-0.770,0.097,0.546,0.554,0.684,0.0,0.0
5,6,2,5,3,2,3,-0.169,-0.726,0.370,0.951,0.684,1.0,0.0


### Generate a cumulative pearson's correlation score for each additional feature against the original 5 features ('Risk', 'NoFaceContact', 'Sick', 'HandSanit', 'HndWshQual'). Also remove columns 'Student', 'Flu', and 'Female'

In [ ]:
df_additional = df_additional.drop(['Student', 'Flu', 'Female'], axis=1)
df_additional = df_additional.dropna()
df_additional.head()

,Vaccin,HndWshFreq,SociDist,RespEttiqu,PersnDist,Complications,Barriers,Inefficacy,KnowlTrans,KnowlMgmt
0,3,4,2,5,1,-1.453,0.000,0.929,-0.554,0.000
1,2,4,5,5,4,0.000,-0.489,0.149,-0.554,1.482
2,3,2,2,2,2,-0.575,-0.234,0.693,-0.182,-1.482
4,2,3,3,5,5,-0.770,0.097,0.546,0.554,0.684
5,2,5,3,2,3,-0.169,-0.726,0.370,0.951,0.684


### Perform cumulative perason correlation for the to-be-selected features with the original features by summing the individual pearson's correlation scores

In [ ]:
from scipy.stats import pearsonr

# Compute cumulative pearson correlation coefficient for each of the additinal features to determine which one is least correlated
# with the already selected features---cumulative pearson for an additional feature is computed as the pearson against
# all the selected features and their sum

def computeCumulativePearson(originalFeatures, additionalFeatures):
  pearson_cumulative = {}
  for column in additionalFeatures:
    sum_pearson = 0.0
    for original_feature in originalFeatures:
      corr, _ = pearsonr(df[column], df[original_feature])
      sum_pearson += corr
    pearson_cumulative[column] = sum_pearson

  print(pearson_cumulative)
  return pearson_cumulative

In [ ]:
originalFeatures = ['Risk', 'NoFaceContact', 'Sick', 'HandSanit', 'HndWshQual']
additionalFeatures = list(df_additional.columns)

pearson_cumulative = computeCumulativePearson(originalFeatures, additionalFeatures)

{'Vaccin': 0.1435758183202581, 'HndWshFreq': 0.5378926251165679, 'SociDist': 0.4345271810521063, 'RespEttiqu': 0.030453488240095333, 'PersnDist': 0.5255863945245257, 'Complications': 0.39507329536695224, 'Barriers': -0.11403416001302383, 'Inefficacy': -0.3764013827291236, 'KnowlTrans': -0.19234972280409235, 'KnowlMgmt': -0.12491632635484362}


### Determine the least correlated feature in pearson 

In [ ]:
target = 0
min_corr_feature, value = min(pearson_cumulative.items(), key=lambda kv : abs(kv[1] - target))

print(min_corr_feature)

RespEttiqu


### Thus, we use feature 'RespEttiqu' as an additional feature

In [ ]:
# Features to extract
new_features = ['Risk', 'NoFaceContact', 'Sick', 'HandSanit', 'HndWshQual', min_corr_feature]   # change with the required features

df_new = _extractFeatures(new_features)  # the raw data to perform fuzzy c-means clustering on before standardizing

### Run the fuzzy c-means clustering for the newly formed dataframe

In [ ]:
F2 = FuzzyCMeans(2, 2, df_cluster, 0.0005)   # Setting number of clusters to 2 and fuzzification parameter to 2
cluster_dict_new, centroid_dict_new = F1._fuzzyCMeans()

In [ ]:
# Extract the records along with their cluster membership and their weights
record_dict_new = F2._getRecordAndWeights(cluster_dict_new)    # this data structure will have the form {cluster_id:[record-1, record-2,..., record-m]}

In [ ]:
print(record_dict_new[0][4])
print(record_dict_new[1][4])

0.4977385226892457
0.5022614773107543


In [ ]:
sse = F2._fuzzyCMeansSSE(cluster_dict_new, centroid_dict_new)
print(sse)

370.94848794231234


In [ ]:
print(_computeDunnIndex(record_dict_new, centroid_dict_new))

0.010983756709198421
